## This notebook is a short introduction to keras and TF 2.0

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
/
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
import tensorflow as tf

import cProfile

### Basic of TF 2.0

In [2]:
tf.executing_eagerly()

True

Run simple matrix calculations using tf

In [3]:
x = [[2.]]

m = tf.matmul(x,x)
print("x matmul x = {}".format(m))

x matmul x = [[4.]]


Creat Tensor

In [4]:
#创建向量
a = tf.constant([[1,2],
                 [3,4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [5]:
#Broadcasting 
b = tf.add(a,1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [6]:
#element-wise multiplication
print(a*b)  

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [7]:
#矩阵乘法
print(tf.matmul(a,b))

tf.Tensor(
[[10 13]
 [22 29]], shape=(2, 2), dtype=int32)


Eager execution works perfectly with Numpy

In [8]:
import numpy as np

In [9]:
c = np.multiply(a,b)
print(c)
print(tf.multiply(a,b))

[[ 2  6]
 [12 20]]
tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [10]:
#Transfer a tensor to numpy array
print(a.numpy())

[[1 2]
 [3 4]]


### Computing gradients

Automatic differentiation is useful for implementing machine learning algorithms such as backpropagation for training neural networks. During eager execution, use tf.GradientTape to trace operations for computing gradients later

In [33]:
#参数初始化
w = tf.Variable([[1.0]])

with tf.GradientTape() as tape:
    loss = w * w
#loss对于w的导数
grad = tape.gradient(loss,w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


### Train a model

In [32]:
#mnist：一个收录了许多 28 x 28 像素手写数字图片（以灰度值矩阵存储）及其对应的数字的数据集
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [33]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(10000, 28, 28)
(10000,)
(1000, 28, 28)
(1000,)


In [34]:
x_train = tf.cast(x_train[...,tf.newaxis]/255, tf.float32),
x_test = tf.cast(x_test[...,tf.newaxis]/255, tf.float32),

#y_train = y_train.astype('float32')
#y_test = y_test.astype('float32')
y_train = tf.keras.utils.to_categorical(y_train,10)
y_test = tf.keras.utils.to_categorical(y_test,10)

In [35]:
#Build the model using Sequential

mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,[3,3],activation='relu',
                          input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64,[3,3],activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation="softmax")
])

In [36]:
mnist_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)              

In [37]:
#Build the model using Model
inputs = tf.keras.Input(shape=(None,None,1),name="digits")
conv_1 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(inputs)
conv_2 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(conv_1)
ave_pool = tf.keras.layers.GlobalAveragePooling2D()(conv_2)
outputs = tf.keras.layers.Dense(10)(ave_pool)
mnist_model_2 = tf.keras.Model(inputs=inputs,outputs=outputs)

In [38]:
mnist_model_2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, None, 16)    160       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
global_average_pooling2d_1 ( (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                170       
Total params: 2,650
Trainable params: 2,650
Non-trainable params: 0
_________________________________________________________________


### Two training methods

##### Use keras fit method

In [39]:
mnist_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
#validation_split=0.1,shuffle=True,
loss = tf.keras.losses.categorical_crossentropy,
metrics = ["accuracy"])

In [40]:
mnist_model_2.compile(loss = tf.keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [41]:
#mnist_model.fit(x_train,y_train,batch_size=128,epochs=3)
mnist_model.fit(x_train,y_train,batch_size=128,epochs=3,validation_split=0.1,shuffle=True)

Train on 9000 samples, validate on 1000 samples
Epoch 1/3
9000/9000 [==============================] - 11s 1ms/sample - loss: 2.2982 - accuracy: 0.1137 - val_loss: 2.2777 - val_accuracy: 0.2320
Epoch 2/3
9000/9000 [==============================] - 11s 1ms/sample - loss: 2.2802 - accuracy: 0.1556 - val_loss: 2.2572 - val_accuracy: 0.3270
Epoch 3/3
9000/9000 [==============================] - 10s 1ms/sample - loss: 2.2607 - accuracy: 0.1929 - val_loss: 2.2351 - val_accuracy: 0.3940


In [42]:
mnist_model.evaluate(x_test,y_test)

992/1000 [============================>.] - ETA: 0s - loss: 2.2491 - accuracy: 0.341000/1000 [==============================] - 0s 350us/sample - loss: 2.2489 - accuracy: 0.3480


[2.2488721389770507, 0.348]

In [46]:
import matplotlib.pyplot as plt
print(y_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [42]:
x_train[0]

<tf.Tensor: shape=(10000, 28, 28, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
  

In [44]:
x_test[0]

<tf.Tensor: shape=(1000, 28, 28, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
   

In [48]:
a = mnist_model.predict(x_test[0])
#mnist_model.predict(x_test[0][1])
#plt.imshow(a)
print(a)

[[0.10135431 0.09804783 0.10152224 ... 0.11481234 0.10096681 0.10192592]
 [0.10944591 0.09564453 0.10643231 ... 0.08838054 0.09809607 0.09522364]
 [0.1027506  0.10489816 0.09899054 ... 0.10480834 0.09822866 0.09588618]
 ...
 [0.13192183 0.08209091 0.10611272 ... 0.10968053 0.08407021 0.10759906]
 [0.11754117 0.08997462 0.11115732 ... 0.108767   0.08779496 0.10086074]
 [0.10390064 0.09075917 0.11259439 ... 0.09913587 0.10047153 0.10791851]]


### Use TF 2.0 

In [49]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [50]:
dataset = tf.data.Dataset.from_tensor_slices(
(tf.cast(x_train[...,tf.newaxis]/255, tf.float32),
 tf.cast(y_train,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [49]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_history = []

In [50]:
for epoch in range(5):
    
    for (batch, (images,labels)) in enumerate (dataset):
        
        with tf.GradientTape() as tape:
            
            logits = mnist_model(images,training=True)
            loss_value = loss(labels,logits)
            
        grads = tape.gradient(loss_value,mnist_model.trainable_variables)
        optimizer.apply_gradients(zip(grads,mnist_model.trainable_variables))
        
    print("Epoch {} finishted".format(epoch))

Epoch 0 finishted
Epoch 1 finishted
Epoch 2 finishted
Epoch 3 finishted
Epoch 4 finishted
